In [2]:
# !pip install astropy
import rebound
import reboundx
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
from multiprocess import Pool

In [7]:
date='2024-01-01 00:00'
quad = rebound.Simulation()
quad.add('Sun', date=date, hash='sun')
quad.add('Mercury', date=date)
quad.add('Venus', date=date)
quad.add('Earth', date=date, hash='earth')
quad.add('Mars', date=date)
quad.add('Jupiter', date=date)
quad.add('Saturn', date=date)
quad.add('Uranus', date=date)
quad.add('Neptune', date=date)
quad.move_to_com()
quad.convert_particle_units('AU', 'year', 'Msun')
qps = quad.particles

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Mercury'... 
Found: Mercury Barycenter (199) (chosen from query 'Mercury')
Searching NASA Horizons for 'Venus'... 
Found: Venus Barycenter (299) (chosen from query 'Venus')
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mars'... 
Found: Mars Barycenter (4) (chosen from query 'Mars')
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')


In [8]:
quad.integrator = "whfast"
quad.save_to_file("quad.bin", interval=1e4,delete_file=True)
# quad.dt = (365.25*0.05)/365.25
quad.dt = np.sqrt(10)/365.25
quad.exit_min_distance = 1e-3 # ~4x the distance between the Earth and Moon.
quad.exit_max_distance = 1000. # 1000 AU, Unlikely that a planet is bound.

rebx = reboundx.Extras(quad)

# add GR
gr = rebx.load_force('gr_potential')
rebx.add_force(gr)
gr.params['c'] = 63240 # speed of light in AU/yr

cf = rebx.load_force("quadrupole")
rebx.add_force(cf)

earth_m = qps['earth'].m/1.012
f = 0.8525
mu_eff = f*(1*0.012*(earth_m)**2)/(1.012*earth_m)

A = -1 * 3 * quad.G * qps['sun'].m / (4 * qps['earth'].m)
R = 0.0025696

qps['earth'].params["Rcentral"] = R
qps['earth'].params["mu_effcentral"] = mu_eff

In [ ]:
%%time
quad.integrate(-3*1e7)